In [ ]:
import pandas as pd
data = pd.read_csv('force_Re150.txt', delim_whitespace=True, header=None, names=['Time', 'Cl', 'Cd'])
data_filtered = data.iloc[249:]
data_filtered.to_csv('force_Re150_filtered.csv', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('force_Re150_filtered.csv')
df = df[df['Time']>=250]
X = df.iloc[:, 1]
X.shape
print(X.min())
print(X.max())
X = df.iloc[:, 1]

X_centered = X - X.mean()
print(X_centered.min())
print(X_centered.max())
Normalized_X = X_centered.to_numpy()
Cl = Normalized_X
print(Cl)


1.36259
1.419083
-0.028361293121365705
0.02813170687863442
[0.02810371 0.02810271 0.02810171 ... 0.02795471 0.02793471 0.02791071]


In [ ]:
Cl = Normalized_X.T
dt = 0.03


Cl_dot = np.gradient(Cl, dt)


X_train = np.vstack([Cl, Cl_dot]).T

differentiation_method = ps.FiniteDifference(order=2)


#polynomial_library = ps.PolynomialLibrary(degree = 6)
#polynomial_library = ps.FourierLibrary(n_frequencies = 2)

lib = [lambda Cl: Cl_dot, lambda Cl: Cl_dot**3, lambda Cl: Cl]
library_function_names = [lambda x: Cl_dot, lambda x: Cl_dot**3, lambda x: Cl]
custom_library = ps.CustomLibrary(library_functions = lib, function_names = library_function_names)


optimizer = ps.STLSQ(threshold=0.01)


feature_names = ['Cl', 'Cl_dot']


model = ps.SINDy(feature_names=feature_names,
                 optimizer=optimizer,
                 differentiation_method=differentiation_method,
                 feature_library=custom_library)


model.fit(X_train, t=dt)


model.print()


(Cl)' = 0.333 [-3.33333333e-05 -3.33333333e-05 -6.66666667e-05 ... -6.50000000e-04
 -7.33333333e-04 -8.00000000e-04] + 0.333 [-3.33333333e-05 -3.33333333e-05 -6.66666667e-05 ... -6.50000000e-04
 -7.33333333e-04 -8.00000000e-04] + 0.333 [0.02810371 0.02810271 0.02810171 ... 0.02795471 0.02793471 0.02791071]
(Cl_dot)' = -0.057 [0.02810371 0.02810271 0.02810171 ... 0.02795471 0.02793471 0.02791071]


In [ ]:
import numpy as np
import pysindy as ps
from sklearn.preprocessing import StandardScaler

# Assuming Normalized_X is your Cl data
Cl = Normalized_X.T
dt = 0.03

# Compute the derivatives
Cl_dot = np.gradient(Cl, dt)
ddCl = np.gradient(Cl_dot, dt)

# Prepare the data for SINDy
X_train = np.vstack([Cl, Cl_dot]).T

# Polynomial library with degree 3
polynomial_library = ps.PolynomialLibrary(degree=3)

# Scale the data if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Fit the SINDy model
optimizer = ps.STLSQ(threshold=0.01)
model = ps.SINDy(feature_names=['Cl', 'Cl_dot'],
                 optimizer=optimizer,
                 differentiation_method=ps.FiniteDifference(order=3),
                 feature_library=polynomial_library)

model.fit(X_train_scaled, t=dt)
model.print()

# Print coefficients for interpretation
coefficients = model.coefficients()
print(coefficients)


(Cl)' = 0.239 Cl_dot
(Cl_dot)' = 0.169 1 + 0.119 Cl + 1.536 Cl_dot + -0.088 Cl^2 + -0.021 Cl Cl_dot + -0.064 Cl_dot^2 + -0.180 Cl^3 + -0.769 Cl^2 Cl_dot + -0.180 Cl Cl_dot^2 + -0.766 Cl_dot^3
[[ 0.          0.          0.23929524  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.16875574  0.1187187   1.53592263 -0.08829272 -0.02124782 -0.06360506
  -0.17950942 -0.76889592 -0.18048769 -0.76604301]]


In [ ]:
import numpy as np
import pysindy as ps
from sklearn.preprocessing import StandardScaler


Cl = Normalized_X.T
dt = 0.003


Cl_dot = np.gradient(Cl, dt)



X_train = np.vstack([Cl, Cl_dot]).T


polynomial_library = ps.PolynomialLibrary(degree=5)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


optimizer = ps.STLSQ(threshold=1e-2)
model = ps.SINDy(feature_names=['Cl', 'Cl_dot'],
                 optimizer=optimizer,
                 differentiation_method=ps.FiniteDifference(order=2),
                 feature_library=polynomial_library)

model.fit(X_train_scaled, t=dt)
model.print()


coefficients = model.coefficients()
print(coefficients)


(Cl)' = 2.393 Cl_dot
(Cl_dot)' = -5.112 1 + 4.039 Cl + 2.716 Cl_dot + 5.030 Cl^2 + -0.218 Cl Cl_dot + 5.238 Cl_dot^2 + -1.359 Cl^3 + -2.744 Cl^2 Cl_dot + -7.253 Cl Cl_dot^2 + -2.816 Cl_dot^3 + -1.259 Cl^4 + 0.097 Cl^3 Cl_dot + -2.597 Cl^2 Cl_dot^2 + 0.083 Cl Cl_dot^3 + -1.350 Cl_dot^4 + -0.929 Cl^5 + 0.692 Cl^4 Cl_dot + 1.093 Cl^3 Cl_dot^2 + 1.425 Cl^2 Cl_dot^3 + 2.012 Cl Cl_dot^4 + 0.730 Cl_dot^5
[[ 0.00000000e+00  0.00000000e+00  2.39318786e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.41360855e-04
   0.00000000e+00 -1.74515636e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.45608435e-06  0.00000000e+00  2.28619701e-05  0.00000000e+00
   1.98923905e-05]
 [-5.11207908e+00  4.03934201e+00  2.71583184e+00  5.02987552e+00
  -2.17835730e-01  5.23769340e+00 -1.35914494e+00 -2.74387834e+00
  -7.25294486e+00 -2.81627474e+00 -1.25857305e+00  9.68736476e-02
  -2.59726461e+00  8.32694999e-02 -1.34990802e+00 -9

In [ ]:
import numpy as np
import pysindy as ps
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

# Assuming Normalized_X is your Cl data
Cl = Normalized_X.T
dt = 0.003

# Compute the derivatives
Cl_dot = np.gradient(Cl, dt)
ddCl = np.gradient(Cl_dot, dt)

# Prepare the data for SINDy
X_train = np.vstack([Cl, Cl_dot]).T

# Polynomial library with degree 3
polynomial_library = ps.PolynomialLibrary(degree=3)

# Scale the data if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Create and fit the model
lasso_alpha = 0.001
# Adjust this value as needed
lasso_optimizer = ps.STLSQ(threshold=lasso_alpha)

model = ps.SINDy(feature_names=['Cl', 'Cl_dot'],
                 optimizer=lasso_optimizer,
                 differentiation_method=ps.FiniteDifference(order=2),
                 feature_library=polynomial_library)

model.fit(X_train_scaled, t=dt)

# Print results
model.print()

# Print coefficients for interpretation
coefficients = model.coefficients()
print(coefficients)


(Cl)' = 2.393 Cl_dot
(Cl_dot)' = 0.032 1 + 1.009 Cl + -0.037 Cl_dot + -0.020 Cl^2 + -0.032 Cl Cl_dot + -0.043 Cl_dot^2 + -1.701 Cl^3 + 0.018 Cl^2 Cl_dot + -1.703 Cl Cl_dot^2 + 0.019 Cl_dot^3
[[ 0.          0.          2.39291819  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.0320494   1.00906876 -0.03708725 -0.02011479 -0.03246118 -0.04276429
  -1.70101056  0.01821208 -1.70267436  0.01902039]]
